Split bounding boxes into new images

In [ ]:
import cv2
import json
import os
import numpy as np

# Load annotations
with open("/Users/rfalcao/Documents/FYP/Cusp Images_081224/boxes_2.json") as f:
    data = json.load(f)

image_dir = "/Users/rfalcao/Documents/FYP/Cusp Images_081224/Clear"
output_dir = "/Users/rfalcao/Documents/FYP/Cusp Images_081224/Boxed"

os.makedirs(output_dir, exist_ok=True)

for img_info in data["images"]:
    img_path = os.path.join(image_dir, img_info["file_name"])
    img = cv2.imread(img_path)

    if img is None:
        print(f"Could not read image: {img_info['file_name']}")
        continue

    for ann in data["annotations"]:
        if ann["image_id"] != img_info["id"]:
            continue

        x, y, w, h = ann["bbox"]
        angle = ann["attributes"].get("rotation", 0)

        center = (x + w / 2, y + h / 2)

        # Get rotation matrix
        M = cv2.getRotationMatrix2D(center, angle, 1.0)

        # Warp the image (rotate around the bbox center)
        rotated = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]), flags=cv2.INTER_CUBIC)

        # Now crop the box from the rotated image
        x1 = int(center[0] - w / 2)
        y1 = int(center[1] - h / 2)
        x2 = int(center[0] + w / 2)
        y2 = int(center[1] + h / 2)

        # Clamp to image boundaries
        x1 = max(0, x1)
        y1 = max(0, y1)
        x2 = min(rotated.shape[1], x2)
        y2 = min(rotated.shape[0], y2)

        if x2 > x1 and y2 > y1:
            cropped = rotated[y1:y2, x1:x2]
            save_path = os.path.join(output_dir, f"cusp_{ann['id']}.png")
            cv2.imwrite(save_path, cropped)
        else:
            print(f"Invalid rotated crop for annotation {ann['id']} in {img_info['file_name']}")


In [ ]:
import cv2
import json
import os
import numpy as np
import matplotlib.pyplot as plt

# Load annotations
with open("/Users/rfalcao/Documents/FYP/Cusp Images_081224/boxes_2.json") as f:
    data = json.load(f)

image_dir = "/Users/rfalcao/Documents/FYP/Cusp Images_081224/Clear"

# Loop through images
for img_info in data["images"]:
    img_path = os.path.join(image_dir, img_info["file_name"])
    img = cv2.imread(img_path)

    if img is None:
        print(f"Could not read image: {img_info['file_name']}")
        continue

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB for matplotlib
    img_copy = img_rgb.copy()

    for ann in data["annotations"]:
        if ann["image_id"] != img_info["id"]:
            continue

        x, y, w, h = ann["bbox"]
        angle = ann["attributes"].get("rotation", 0)

        center = (x + w / 2, y + h / 2)
        size = (w, h)

        # Create rotated rectangle and get 4 points
        rect = (center, size, angle)
        box = cv2.boxPoints(rect)
        box = np.intp(box)

        # Draw the rotated box
        cv2.drawContours(img_copy, [box], 0, (0, 255, 0), 2)
        cv2.putText(img_copy, f"ID {ann['id']}", (int(x), int(y - 10)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

    # Display using matplotlib
    plt.figure(figsize=(10, 10))
    plt.imshow(img_copy)
    plt.title(f"Image: {img_info['file_name']}")
    plt.axis('off')
    plt.show()

    # Optional: break after showing a few images
    # break  # Uncomment to preview only the first one


Separate Images into new categories, assuming you have non cusp examples

In [ ]:
import os
import json
import shutil

# Paths
image_dir = "/Users/rfalcao/Documents/FYP/Cusp Images_081224/Clear"  # Folder containing all images
annotation_file = "/Users/rfalcao/Documents/FYP/Cusp Images_081224/instances_default.json"  # Exported JSON file
output_dir = "/Users/rfalcao/Documents/FYP/Cusp Images_081224"

# Create output folders
os.makedirs(f"{output_dir}/cusp_images", exist_ok=True)
os.makedirs(f"{output_dir}/non_cusp_images", exist_ok=True)
os.makedirs(f"{output_dir}/unknown_images", exist_ok=True)

# Load annotation file
with open(annotation_file) as f:
    data = json.load(f)

# Get images with annotations
annotated_images = {}
for ann in data["annotations"]:
    img_id = ann["image_id"]
    if img_id not in annotated_images:
        annotated_images[img_id] = []
    annotated_images[img_id].append(ann["segmentation"])

# Get image file names
image_info = {img["id"]: img["file_name"] for img in data["images"]}

# Process each image
for img_id, filename in image_info.items():
    src_path = os.path.join(image_dir, filename)
    if os.path.exists(src_path):
        if img_id in annotated_images and annotated_images[img_id]:  # Has annotations
            dest_folder = "cusp_images"
        else:  # No annotations
            dest_folder = "non_cusp_images"
        shutil.move(src_path, os.path.join(output_dir, dest_folder, filename))
    else:
        print(f"Warning: {filename} is missing from {image_dir}")

# Move any leftover images (not in JSON)
for filename in os.listdir(image_dir):
    src_path = os.path.join(image_dir, filename)
    dest_path = os.path.join(output_dir, "unknown_images", filename)
    shutil.move(src_path, dest_path)

print("Images sorted successfully!")


In [ ]:
from pycocotools.coco import COCO
import numpy as np
import cv2
import os

# Paths
annotation_file = "/Users/rfalcao/Documents/FYP/Cusp Images_081224/Run_1/instances_default.json"
image_dir = "/Users/rfalcao/Documents/FYP/Cusp Images_081224/Run_1/cusp_images"
mask_output_dir = "/Users/rfalcao/Documents/FYP/Cusp Images_081224/Run_1/masks"
untagged_image_dir = "/Users/rfalcao/Documents/FYPCusp Images_081224/Run_1/non_cusp_images"

os.makedirs(mask_output_dir, exist_ok=True)

# Load COCO JSON
coco = COCO(annotation_file)

# Process images
for img_id in coco.getImgIds():
    img_info = coco.loadImgs(img_id)[0]
    img_filename = img_info["file_name"]

    # Ensure proper mask filename (change .jpg to .png)
    # mask_filename = os.path.splitext(img_filename)[0] + ".png"
    mask_filepath = os.path.join(mask_output_dir, mask_filename)

    # Try loading from cusp images folder
    img_path = os.path.join(image_dir, img_filename)
    if not os.path.exists(img_path):
        img_path = os.path.join(untagged_image_dir, img_filename)

    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

    # Skip missing images
    if img is None:
        print(f"Skipping {img_filename} (not found in any folder)")
        continue

    # Create an empty mask
    mask = np.zeros(img.shape, dtype=np.uint8)

    # Get annotations (if any)
    ann_ids = coco.getAnnIds(imgIds=img_id)
    if ann_ids:
        anns = coco.loadAnns(ann_ids)
        for ann in anns:
            segmentation = ann["segmentation"]
            for seg in segmentation:
                points = np.array(seg, dtype=np.int32).reshape(-1, 2)
                cv2.fillPoly(mask, [points], 255)  # Fill the polygon mask

    # Save mask as PNG (lossless format)
    cv2.imwrite(mask_filepath, mask)

print("✅ Masks extracted and saved as PNGs!")



In [ ]:
import os
import cv2
import numpy as np

# Paths

padded_image_dir = "/Users/rfalcao/Documents/FYP/ManualSegmentationAnns/padded/images"
padded_mask_dir = "/Users/rfalcao/Documents/FYP/ManualSegmentationAnns/padded/masks"

image_dir = "/Users/rfalcao/Documents/FYP/ManualSegmentationAnns/cusp_images"
mask_dir = "/Users/rfalcao/Documents/FYP/ManualSegmentationAnns/masks"

os.makedirs(padded_image_dir, exist_ok=True)
os.makedirs(padded_mask_dir, exist_ok=True)

# Step 1: Find max width & height in dataset
max_width, max_height = 0, 0
for filename in os.listdir(image_dir):
    img_path = os.path.join(image_dir, filename)
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if img is not None:
        h, w = img.shape
        max_width = max(max_width, w)
        max_height = max(max_height, h)

print(f"Max dimensions for padding: {max_width}x{max_height}")

# Step 2: Pad each image & mask
for filename in os.listdir(image_dir):
    img_path = os.path.join(image_dir, filename)
    mask_path = os.path.join(mask_dir, filename)

    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    if img is None or mask is None:
        print(f"Skipping {filename} (missing image or mask)")
        continue

    h, w = img.shape
    pad_top = (max_height - h) // 2
    pad_bottom = max_height - h - pad_top
    pad_left = (max_width - w) // 2
    pad_right = max_width - w - pad_left

    # Apply same padding to image and mask
    padded_img = cv2.copyMakeBorder(img, pad_top, pad_bottom, pad_left, pad_right, cv2.BORDER_CONSTANT, value=0)
    padded_mask = cv2.copyMakeBorder(mask, pad_top, pad_bottom, pad_left, pad_right, cv2.BORDER_CONSTANT, value=0)

    # Save results
    cv2.imwrite(os.path.join(padded_image_dir, filename), padded_img)
    cv2.imwrite(os.path.join(padded_mask_dir, filename), padded_mask)

print("✅ Padding complete! All cusp images and masks are now the same size.")
